In [9]:

# Force keras to use the CPU becuase it's actually faster for this size network
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import numpy as np
import pandas as pd

from numpy import cos, sin, pi
import matplotlib.pyplot as plt
import scipy.integrate as integrate

%matplotlib inline
import matplotlib.animation as animation

from IPython.display import HTML

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.constraints import max_norm



# TODO should move all this to a module (probably merge with the old cartpole module we were using)

# ball: m = 1 kg
# pole: L = 1 m
# cart: M = 4 kg

g = 9.81  # m/s^2
l1 = 1
l2 = 1
lG1 = .5
lG2 = .5


rad = np.pi / 180
u_hist = []


#animation generation
def animate_pend(y):
    th = y[:,0]
    x1 = y[:,1]
    y1 = 0.0

    x2 = L * sin(th) + x1
    y2 = -L * cos(th) + y1

    fig = plt.figure()
    ax = fig.add_subplot(111, autoscale_on=False, aspect='equal', \
                     xlim=(-3, 3), ylim=(-3, 3))
    ax.grid()

    line, = ax.plot([], [], 'o-', lw=2)
    time_template = 'time = %.1fs'
    time_text = ax.text(0.05, 0.9, '', transform=ax.transAxes)


    def init():
        line.set_data([], [])
        time_text.set_text('')
        return line, time_text


    def animate(i):
        thisx = [x1[i], x2[i]]
        thisy = [y1, y2[i]]

        line.set_data(thisx, thisy)
        time_text.set_text(time_template % (i * dt))
        return line, time_text


    return animation.FuncAnimation(fig, animate, np.arange(1, len(y)), interval=40, blit=True, init_func=init)



# control input: u = F
# q is the state vector
def control(q):
    if (q[0] < 140 * rad) or (q[0] > 220 * rad):
        # swing up
        # energy error: Ee
        Ee = 0.5 * m * L * L * q[2] ** 2 - m * g * L * (1 + cos(q[0]))
        # energy control gain:
        k = 0.23
        # input acceleration: A (of cart)
        A = k * Ee * cos(q[0]) * q[2]
        # convert A to u (using EOM)
        delta = m * sin(q[0]) ** 2 + M
        u = A * delta - m * L * (q[2] ** 2) * sin(q[0]) - m * g * sin(q[2]) * cos(q[2])
    else:
        # balancing
        # LQR: K values from MATLAB
        k1 = 140.560
        k2 = -3.162
        k3 = 41.772
        k4 = -8.314
        u = -(k1 * (q[0] - pi) + k2 * q[1] + k3 * q[2] + k4 * q[3])
        
    return u


# state vector: q = transpose([theta, x, d(theta)/dt, dx/dt])
def derivs(q, t):
    #TODO should definitley move this outside of python, would be a nice excuse to learn how to write a C module that interfaces with python
    dqdt = np.zeros_like(q)

    # control input
    u = control(q,t)

    dq1 = (l1* Lg2**2 * M2**2 * cos(q[0] - q[1])*( g*cos(q[1]) - q[2]**2*l1*sin(q[0] - q[1])) \
           - (J2 + lG2**2*M2)*(-u + g*lG1*M1 + l1*M2)*cos(q[0] + q[3]**2 *l1*lG2*M2*sin(q[0] - q[1]))) \
            / (-(J1 + l1^2*M2)*(J2 + Lg2**2 * M2) + l1**2 * lG2 **2 * M2**2 * cos(q[0] - q[1])**2)
               
    
    dq2 = (2 * lG2 * M2 * ((J1 + l1**2 * M2) * (-g * cos(q[1]) + q[2]**2 * l1 * sin(q[0] - q[1]) + \
           l1 * cos(q[0] - q[1]) (-u + g * (lG1 * M1 + l1 * M2) * cos(q[1]) + \
            q[3]**2 * l1 * lG2 * M2 * sin(q[0] - q[1])))/(-2 * J1 * (J2 + lG2**2 * M2) - \
             l1**2 * M2 * (2 * J2 + lG2**2 * M2) + l1**2 * lG2**2 * M2**2 * cos(2 (q[0] - q[1]))))) 

    return dqdt

In [ ]:
# time step
dt = 0.1
t = np.arange(0.0, 20, dt)

rad = np.pi / 180


num_trials = 1
num_states = 4
num_t = len(t)
y = np.zeros((num_t, num_states, num_trials))


for i in range(num_trials):
    # initial conditions
    theta = 0
    x = 0.0
    th_dot = 2*(i/num_trials) - 1  # an initial velocity, triggers the swing up control
    xdot = 0.0

    # initial state
    state = np.array([theta, x, th_dot, xdot])

    # integrate the ODE using scipy.integrate.
    u_hist = []
    y[:, :, i] = integrate.odeint(derivs, state, t)


u = np.zeros((num_t, num_trials))
for i in range(num_trials):
    for t in range(num_t):
        u[t,i] = control(y[t,:,i]) 

        
ani = animate_pend(y[:,:,0])
HTML(ani.to_html5_video())

In [ ]:
# Create our LSTM network

look_back = 1

with tf.variable_scope('pi/pol/'):
    model = Sequential()
    model.add(LSTM(12, input_shape=(4,look_back)))
    #model.add(LSTM(4, input_shape =(12,look_back)))
    model.add(Dense(1))

    #model.add(Dense(2))
    

model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
history = model.fit(y, u, epochs=500, verbose=0)
%matplotlib inline
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

In [ ]:
# This is a closure that returns our NN controller
def make_controller(model):
    def nn_controller(q):
        if (q[0] < 140 * rad) or (q[0] > 220 * rad):
            return model.predict(q.reshape((1,4,1)))
        else:
            # balancing
            # LQR: K values from MATLAB
            k1 = 140.560
            k2 = -3.162
            k3 = 41.772
            k4 = -8.314
            u = -(k1 * (q[0] - pi) + k2 * q[1] + k3 * q[2] + k4 * q[3])
            return u
        
    return nn_controller

In [ ]:
control = make_controller(model)

# initial conditions
theta = 0
x = 0
th_dot = .01 # an initial velocity, triggers the swing up control
xdot = 0.0
t = np.arange(0.0, 20, dt)


# initial state
state = np.array([theta, x, th_dot, xdot])

# integrate the ODE using scipy.integrate.
y_test = integrate.odeint(derivs, state, t)

#ani = animate_pend(y_test)
#HTML(ani.to_html5_video())

In [ ]:
def run_swingup():
    control = make_controller(model)
    
    # initial conditions
    theta = 0
    x = 0
    th_dot = .01 # an initial velocity, triggers the swing up control
    xdot = 0.0
    t = np.arange(0.0, 20, dt)


    # initial state
    state = np.array([theta, x, th_dot, xdot])

    # integrate the ODE using scipy.integrate.
    y_test = integrate.odeint(derivs, state, t)

    #ani = animate_pend(y_test)
    #HTML(ani.to_html5_video())

In [ ]:
control = make_controller(model)

# initial conditions
theta = 0
x = 0
th_dot = .01 # an initial velocity, triggers the swing up control
xdot = 0.0
t_index = np.arange(0.0, 2, dt)


# initial state
state = np.array([theta, x, th_dot, xdot])

# integrate the ODE using scipy.integrate.
# Fill in our u after the fact..
y_test = integrate.odeint(derivs, state, t_index)
u_test = np.zeros((y.shape[0],1))
for t in range(num_t):
        u_test[t] = control(y[t]) 

        
data = np.concatenate((y_test, u_test),axis=1)
names = ['theta','x','thetadot','xdot','u']
yf = pd.DataFrame(data = data, index = t_index, columns = names)


ani = animate_pend(y_test)
HTML(ani.to_html5_video())

In [ ]:
plt.plot(yf['state','theta'])
old_theta = yf['state','theta']


In [ ]:
new_theta = yf['theta']

In [ ]:
np.all(old_theta == new_theta)

In [ ]:
N_train = 1000
from numpy.random import choice
one_indexes = choice(a=N_train, size=int(N_train / 2), replace=False)
X_train[one_indexes, 0] = 1  # very long term memory.

In [ ]:
# messing with multiindexing 
# TODO probably move this to the misc folder, preferably as a more complete example than this one.

names = [
    ('state', 'theta'),
    ('state', 'x'),
    ('state', 'thetadot'),
    ('state', 'xdot'),
    ('control' ,'u')
]

yf = pd.DataFrame(data = data, index = t_index, columns=pd.MultiIndex.from_tuples(names))
